# Import Required Libraries

In [4]:
# pip install spacy

In [5]:
# python -m spacy download en_core_web_sm

In [6]:
import pandas as pd
import spacy

In [7]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Load Dataset

In [10]:
# Load the sentimentdataset.csv file into a pandas DataFrame
df = pd.read_csv('CNN_Articles_sentiment.tsv', delimiter='\t')
df.head()

,text,frequency,sentiment,year
0,Bomb squads in Germany successfully defused on...,1,0.50000,2011
1,College footballs topranked LSU Tigers defeate...,1,0.17570,2011
2,Udinese have gone top of the Italian Serie A t...,1,0.60115,2011
3,The Czech Republic have secured their first Fe...,1,0.87530,2011
4,Cristiano Ronaldo scored a superb hattrick as ...,1,0.92595,2011


# Preprocess dataset

## Exctract named entities

In [11]:
# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    return ', '.join([ent.text for ent in doc.ents])

# Apply the function to the 'text' column
df['important_words'] = df['text'].apply(extract_entities)

In [12]:
df.head(10)

,text,frequency,sentiment,year,important_words
0,Bomb squads in Germany successfully defused on...,1,0.50000,2011,"Germany, Sunday, two"
1,College footballs topranked LSU Tigers defeate...,1,0.17570,2011,"LSU Tigers, Saturday night"
2,Udinese have gone top of the Italian Serie A t...,1,0.60115,2011,"the Italian Serie, Lazio, Antonio Di Natale, S..."
3,The Czech Republic have secured their first Fe...,1,0.87530,2011,"first, Fed Cup, Lucie Hradecka, Kveta Peschke,..."
4,Cristiano Ronaldo scored a superb hattrick as ...,1,0.92595,2011,"Cristiano Ronaldo, Real Madrid, Osasuna, the S..."
5,Bayern Munich have moved five points clear at ...,1,0.76335,2011,"Bayern Munich, five, German, Bundesliga, Augsb..."
6,World number six Martin Kaymer carded a superb...,1,0.95590,2011,"six, Martin Kaymer, the HSBC Champions, Shangh..."
7,Authorities have issued an arrest warrant for ...,1,0.27060,2011,"Dallas, Terrell Owens, last month"
8,Third seed Roger Federer returned to winning w...,1,0.93600,2011,"Third, Roger Federer, a month, Japans, Swiss, ..."
9,Former Serb international midfielder Sinisa Mi...,1,0.27060,2011,"Serb, Sinisa Mihajlovic, Serie A, Fiorentina, ..."


## Split important words

In [13]:
# Remove the 'text' and 'frequency' columns
df = df.drop(columns=['text', 'frequency'])

# Split the 'important_words' column into multiple columns
important_words_split = df['important_words'].str.split(',', expand=True)

# Rename the new columns to word1, word2, word3, ...
important_words_split.columns = [f'word{i+1}' for i in range(important_words_split.shape[1])]

# Concatenate the original dataframe (without 'text' and 'important_words') with the new split columns
df = pd.concat([df.drop(columns=['important_words']), important_words_split], axis=1)

,frequency,sentiment,year,word1,word2,word3,word4,word5,word6,word7,word8,word9,word10,word11,word12,word13,word14,word15
0,1,0.50000,2011,Germany,Sunday,two,None,None,None,None,None,None,None,None,None,None,None,None
1,1,0.17570,2011,LSU Tigers,Saturday night,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1,0.60115,2011,the Italian Serie,Lazio,Antonio Di Natale,Siena,Sunday,None,None,None,None,None,None,None,None,None,None
3,1,0.87530,2011,first,Fed Cup,Lucie Hradecka,Kveta Peschke,Russia,Moscow,Sunday,None,None,None,None,None,None,None,None
4,1,0.92595,2011,Cristiano Ronaldo,Real Madrid,Osasuna,the Spanish,three,Barcelonas,Athletic Bilbao,None,None,None,None,None,None,None,None


In [15]:
df.head()

,sentiment,year,word1,word2,word3,word4,word5,word6,word7,word8,word9,word10,word11,word12,word13,word14,word15
0,0.50000,2011,Germany,Sunday,two,None,None,None,None,None,None,None,None,None,None,None,None
1,0.17570,2011,LSU Tigers,Saturday night,None,None,None,None,None,None,None,None,None,None,None,None,None
2,0.60115,2011,the Italian Serie,Lazio,Antonio Di Natale,Siena,Sunday,None,None,None,None,None,None,None,None,None,None
3,0.87530,2011,first,Fed Cup,Lucie Hradecka,Kveta Peschke,Russia,Moscow,Sunday,None,None,None,None,None,None,None,None
4,0.92595,2011,Cristiano Ronaldo,Real Madrid,Osasuna,the Spanish,three,Barcelonas,Athletic Bilbao,None,None,None,None,None,None,None,None


## Melt

In [16]:
# Melt the dataframe to have each row correspond to a single 'word'
melted_df = df.melt(id_vars=['sentiment', 'year'], value_vars=[col for col in df.columns if col.startswith('word')], var_name='word_num', value_name='word')

# Drop rows where 'word' is None
melted_df = melted_df.dropna(subset=['word'])

# Drop the 'word_num' column as it is no longer needed
melted_df = melted_df.drop(columns=['word_num'])
melted_df.head()

,sentiment,year,word
0,0.50000,2011,Germany
1,0.17570,2011,LSU Tigers
2,0.60115,2011,the Italian Serie
3,0.87530,2011,first
4,0.92595,2011,Cristiano Ronaldo


In [17]:
# Drop rows where 'word' is None or empty spaces
melted_df = melted_df.dropna(subset=['word'])
melted_df['word'] = melted_df['word'].str.strip()
melted_df = melted_df[melted_df['word'] != '']

In [18]:
# Create a new column 'source' with the value 'cnn' for all rows
melted_df['source'] = 'cnn'
melted_df.head()

,sentiment,year,word,source
0,0.50000,2011,Germany,cnn
1,0.17570,2011,LSU Tigers,cnn
2,0.60115,2011,the Italian Serie,cnn
3,0.87530,2011,first,cnn
4,0.92595,2011,Cristiano Ronaldo,cnn


## Agreggate

In [19]:
# # Aggregate by 'year' and 'word' and count the frequency
# aggregated_df = melted_df.groupby(['year', 'word']).size().reset_index(name='frequency')
# aggregated_df.head()

,year,word,frequency
0,2011,,15
1,2011,A year old,1
2,2011,AC Milan,9
3,2011,AIC,1
4,2011,ATP,1


In [20]:
# Aggregate by 'word' and 'year'
aggregated_df = melted_df.groupby(['word', 'year']).agg(
    frequency=('word', 'size'),
    sentiment=('sentiment', 'mean')
).reset_index()

In [23]:
aggregated_df.head(20)

,word,year,frequency,sentiment
0,,2011,15,0.506300
1,,2012,57,0.520173
2,,2013,81,0.535538
3,,2014,58,0.446820
4,,2015,165,0.520139
5,,2016,136,0.525694
6,,2017,89,0.532804
7,,2018,124,0.511051
8,,2019,160,0.539537
9,,2020,179,0.506749


In [ ]:
# Create a new column 'source' with the value 'cnn' for all rows
aggregated_df['source'] = 'cnn'

# Rename the column 'word' to 'text'
aggregated_df = aggregated_df.rename(columns={'word': 'text'})

# Reorder the columns
aggregated_df = aggregated_df[['source', 'year', 'text', 'sentiment', 'frequency']]

In [25]:
# Drop rows where 'word' is None or empty spaces
aggregated_df = aggregated_df.dropna(subset=['text'])
aggregated_df = aggregated_df[aggregated_df['text'] != '']

In [27]:
aggregated_df.head(50)

,source,year,text,sentiment,frequency
12,cnn,2018,A million,0.500000,1
13,cnn,2021,A million,0.253050,1
14,cnn,2018,A Breeze of Hope Foundation,0.898200,1
15,cnn,2015,A European Court,0.601150,1
16,cnn,2017,A German Life,0.159600,1
17,cnn,2020,A Second U Foundation,0.911250,1
18,cnn,2012,A day,0.500000,1
19,cnn,2013,A day,0.246033,3
20,cnn,2014,A day,0.268125,2
21,cnn,2015,A day,0.420483,3


# Save Preprocessed dataset

In [28]:
# Export the DataFrame to a new .tsv file
aggregated_df.to_csv('CNN_sentiment.tsv', sep='\t', index=False)

In [29]:
# Print the rows with the 10 largest values of 'frequency'
print(aggregated_df.nlargest(10, 'frequency'))

      source  year       text  sentiment  frequency
6937     cnn  2021      Covid   0.504336        394
31655    cnn  2022    Ukraine   0.376770        391
35185    cnn  2021      first   0.553638        379
31460    cnn  2021         US   0.441416        361
25710    cnn  2022    Russian   0.408030        309
31461    cnn  2022         US   0.435581        306
32690    cnn  2021  Wednesday   0.435256        301
35183    cnn  2019      first   0.585516        280
28567    cnn  2021     Sunday   0.520219        274
30444    cnn  2021   Thursday   0.424573        266


In [31]:
# Print all rows with 'text' equal to 'Ukraine' in order by year
ukraine_rows = aggregated_df[aggregated_df['text'] == 'Ukraine'].sort_values(by='year')
print(ukraine_rows)

      source  year     text  sentiment  frequency
31644    cnn  2011  Ukraine   0.672171          7
31645    cnn  2012  Ukraine   0.415269         21
31646    cnn  2013  Ukraine   0.531460          5
31647    cnn  2014  Ukraine   0.367803        173
31648    cnn  2015  Ukraine   0.353852         48
31649    cnn  2016  Ukraine   0.444290         10
31650    cnn  2017  Ukraine   0.238368         11
31651    cnn  2018  Ukraine   0.233319          8
31652    cnn  2019  Ukraine   0.398802         21
31653    cnn  2020  Ukraine   0.538190          5
31654    cnn  2021  Ukraine   0.394085         13
31655    cnn  2022  Ukraine   0.376770        391
